<a href="https://colab.research.google.com/github/rpjena/random_matrix/blob/main/mctr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

def calculate_risk_contribution(weights, returns, volatilities=None, annualizing_factor=252, use_true_pnl=False, daily_weights=None):
    """
    Calculate the marginal and absolute contribution to total portfolio risk.

    Args:
        weights (np.array or pd.DataFrame): Portfolio weights. For constant portfolio, a single array (e.g., [0.333, 0.333, 0.333]).
                                          For true PnL, a DataFrame with daily weights (rows = dates, columns = components).
        returns (pd.DataFrame): Time series of returns for each component (columns = components).
        volatilities (np.array): Annualized volatility estimates for each component (e.g., [0.15, 0.20, 0.10]). Ignored if use_true_pnl=True.
        annualizing_factor (int): Number of periods per year (e.g., 252 for daily returns).
        use_true_pnl (bool): If True, use true PnL and daily weights; if False, use constant weights and scale to volatilities.
        daily_weights (pd.DataFrame): Optional, DataFrame of daily weights (rows = dates, columns = components). Required if use_true_pnl=True.

    Returns:
        dict: Contains portfolio volatility, marginal contributions, and absolute contributions (%).
    """
    # Ensure inputs are numpy arrays or DataFrames
    if isinstance(weights, list):
        weights = np.array(weights)
    returns = pd.DataFrame(returns)

    if use_true_pnl:
        # True PnL with daily position changes
        if daily_weights is None:
            raise ValueError("daily_weights DataFrame required when use_true_pnl=True")
        daily_weights = pd.DataFrame(daily_weights, index=returns.index, columns=returns.columns)

        # Calculate daily portfolio returns
        portfolio_returns = (returns * daily_weights).sum(axis=1)

        # Portfolio volatility (from portfolio returns)
        portfolio_vol = np.sqrt(annualizing_factor) * portfolio_returns.std()

        # Covariance matrix from returns
        cov_matrix = returns.cov() * annualizing_factor

        # Average weights for MCTR approximation
        avg_weights = daily_weights.mean()

        # Marginal contribution to risk (MCTR)
        mctr = np.dot(cov_matrix, avg_weights) / portfolio_vol

        # Absolute contribution to risk (ACTR)
        actr = avg_weights * mctr
        actr_percent = (actr / portfolio_vol) * 100

    else:
        # Constant portfolio with hypothetical or provided returns
        # Calculate covariance matrix from returns
        cov_matrix = returns.cov() * annualizing_factor

        # Scale covariance matrix to match provided volatilities
        if volatilities is not None:
            volatilities = np.array(volatilities)
            cov_vols = np.sqrt(np.diag(cov_matrix))
            scale_factors = volatilities / cov_vols
            scaling_matrix = np.outer(scale_factors, scale_factors)
            cov_matrix = cov_matrix * scaling_matrix

        # Portfolio volatility
        portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

        # Marginal contribution to risk (MCTR)
        mctr = np.dot(cov_matrix, weights) / portfolio_vol

        # Absolute contribution to risk (ACTR)
        actr = weights * mctr
        actr_percent = (actr / portfolio_vol) * 100

    # Results
    result = {
        'portfolio_volatility': portfolio_vol * 100,  # As percentage
        'marginal_contribution_to_risk': mctr * 100,  # As percentage
        'absolute_contribution_to_risk_percent': actr_percent,
        'component_names': returns.columns.tolist()
    }

    return result

# Example usage
if __name__ == "__main__":
    # Hypothetical data for constant portfolio
    np.random.seed(42)  # For reproducibility
    dates = pd.date_range(start='2024-01-01', periods=252, freq='B')
    returns_data = {
        'Asset_A': np.random.normal(0.0005, 0.0094, 252),  # Approx 15% annualized vol
        'Asset_B': np.random.normal(0.0006, 0.0126, 252),  # Approx 20% annualized vol
        'Asset_C': np.random.normal(0.0003, 0.0063, 252)   # Approx 10% annualized vol
    }
    returns = pd.DataFrame(returns_data, index=dates)

    # Portfolio weights (constant, equal weights)
    weights = [1/3, 1/3, 1/3]

    # Provided volatility estimates (annualized, in decimal)
    volatilities = [0.15, 0.20, 0.10]  # 15%, 20%, 10%

    # Calculate risk contributions (constant portfolio)
    result = calculate_risk_contribution(
        weights=weights,
        returns=returns,
        volatilities=volatilities,
        use_true_pnl=False
    )

    # Print results
    print("Constant Portfolio with Hypothetical PnL:")
    print(f"Portfolio Volatility: {result['portfolio_volatility']:.2f}%")
    print("\nMarginal Contribution to Risk (%):")
    for i, asset in enumerate(result['component_names']):
        print(f"{asset}: {result['marginal_contribution_to_risk'][i]:.2f}%")
    print("\nAbsolute Contribution to Risk (% of total risk):")
    for i, asset in enumerate(result['component_names']):
        print(f"{asset}: {result['absolute_contribution_to_risk_percent'][i]:.2f}%")

    # Example for true PnL with daily weights (commented out, uncomment and provide data)
    """
    # Example true PnL with daily weights
    daily_weights_data = {
        'Asset_A': np.random.uniform(0.2, 0.5, 252),  # Random weights
        'Asset_B': np.random.uniform(0.2, 0.5, 252),
        'Asset_C': np.random.uniform(0.2, 0.5, 252)
    }
    daily_weights = pd.DataFrame(daily_weights_data, index=dates)
    daily_weights = daily_weights.div(daily_weights.sum(axis=1), axis=0)  # Normalize to sum to 1

    result_true_pnl = calculate_risk_contribution(
        weights=daily_weights,
        returns=returns,
        volatilities=None,
        use_true_pnl=True,
        daily_weights=daily_weights
    )

    print("\nTrue PnL with Daily Position Changes:")
    print(f"Portfolio Volatility: {result_true_pnl['portfolio_volatility']:.2f}%")
    print("\nMarginal Contribution to Risk (%):")
    for i, asset in enumerate(result_true_pnl['component_names']):
        print(f"{asset}: {result_true_pnl['marginal_contribution_to_risk'][i]:.2f}%")
    print("\nAbsolute Contribution to Risk (% of total risk):")
    for i, asset in enumerate(result_true_pnl['component_names']):
        print(f"{asset}: {result_true_pnl['absolute_contribution_to_risk_percent'][i]:.2f}%")
    """